In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import warnings
warnings.filterwarnings('ignore')

In [38]:
train = pd.read_csv("train.csv")

In [39]:
static_df = train[['merchant_id', 'mcc_id', 'settlement_period', 'working_type', 'merchant_segment']]
train = train[['merchant_id', 'month_id', 'net_payment_count']]
train = train.sort_values(by=['merchant_id', 'month_id']).reset_index(drop=True)
# 
train['month_id'] = pd.to_datetime(train['month_id'], format='%Y%m')

In [40]:
def filter_by_multiple_gaps(df, threshold):
    # month_id sütununu datetime'a çevirme
    
    result_df = pd.DataFrame()  # Sonuçları saklamak için boş bir dataframe
    
    for merchant_id, group in df.groupby('merchant_id'):
        group = group.sort_values(by='month_id')  # Her grubu tarihe göre sıralama
        valid_indices = []  # Geçerli indeksleri saklamak için boş bir liste
        
        prev_date = None
        for index, row in group.iterrows():
            if prev_date is not None:
                date_diff = (row['month_id'] - prev_date).days
                if date_diff < threshold:
                    # Eğer fark belirlenen eşikten küçükse, bu indeksi sakla
                    valid_indices.append(index)
                else:
                    # Eşikten büyük bir fark bulunduğunda, geçerli indeksleri sıfırla ve bu indeksi ekle
                    valid_indices = [index]
            else:
                # İlk satır her zaman geçerli olarak kabul edilir
                valid_indices.append(index)
            prev_date = row['month_id']
        
        # Geçerli indekslere sahip satırları sonuç DataFrame'ine ekle
        result_df = pd.concat([result_df, group.loc[valid_indices]])
    
    return result_df.reset_index(drop=True)

train = filter_by_multiple_gaps(train, 240)

In [41]:
# Son transaction ve ilk gözlem tarihlerini bulma
last_transaction = train.groupby('merchant_id')['month_id'].max()
first_observation = train.groupby('merchant_id')['month_id'].min()

# Model DataFrames
# model_2020_df = train[train['merchant_id'].isin(first_observation[first_observation <= '2020-09-01'].index)]
# model_2021_df = train[train['merchant_id'].isin(first_observation[(first_observation > '2020-09-01') & (first_observation <= '2022-09-01')].index)]
# model_2022_df = train[train['merchant_id'].isin(first_observation[(first_observation > '2021-09-01') & (first_observation <= '2022-09-01')].index)]
model_2023_df = train[train['merchant_id'].isin(first_observation[first_observation > '2022-12-01'].index)]

# Her merchant_id için gözlem sayısını say
merchant_counts = model_2023_df['merchant_id'].value_counts()

# Her merchant_id için en son gözlem tarihini bul
last_observation = model_2023_df.groupby('merchant_id')['month_id'].max()

# Gözlem sayısı 3'ten az olan veya son gözlem tarihi 202301'den düşük olan merchant_id'leri bul
filtered_merchant_ids = merchant_counts[(last_observation <= '2023-06-01')].index
# (merchant_counts <= 1) | 

# Bu merchant_id'leri no_model DataFrame'ine ata
churn_df = model_2023_df[model_2023_df['merchant_id'].isin(filtered_merchant_ids)]

# Diğer verileri updated_train DataFrame'ine ata
model_2023_df = model_2023_df[~model_2023_df['merchant_id'].isin(filtered_merchant_ids)]

# # Her merchant_id için toplam geçme sayısını hesaplama
# merchant_pass_counts = model_2023_df.groupby('merchant_id').size()

# # Geçme sayısı 5'ten az olan merchant_id'leri bulma
# rule_based_merchant_ids = merchant_pass_counts[merchant_pass_counts <= 3].index

# # Bu merchant_id'leri rule_based_df DataFrame'ine ata
# rule_based_df = model_2023_df[model_2023_df['merchant_id'].isin(rule_based_merchant_ids)]

# # Bu merchant_id'leri model_2023_df DataFrame'inden çıkar
# model_2023_df = model_2023_df[~model_2023_df['merchant_id'].isin(rule_based_merchant_ids)]

# Set ve DataFrames'in boyutlarını kontrol etme
churn_df.shape, model_2023_df.shape

((2224, 3), (10215, 3))

In [42]:
# import pandas as pd
# import numpy as np

# def detect_and_cap_outliers(df):
#     df['rolling_mean'] = df['net_payment_count'].rolling(window=3, min_periods=1).mean()
#     df['rolling_std'] = df['net_payment_count'].rolling(window=3, min_periods=1).std()  # ddof=0 for population std
#     df['lower_limit'] = df['rolling_mean'] - 4 * df['rolling_std']
#     df['upper_limit'] = df['rolling_mean'] + 4 * df['rolling_std']
    
#     # Aykırı değerleri tespit
#     df['is_outlier'] = (df['net_payment_count'] < df['lower_limit']) | (df['net_payment_count'] > df['upper_limit'])
    
#     # Aykırı değerleri baskıla
#     df['net_payment_count'] = np.where(df['net_payment_count'] < df['lower_limit'], df['lower_limit'],
#                                               np.where(df['net_payment_count'] > df['upper_limit'], df['upper_limit'],
#                                                        df['net_payment_count']))
#     return df

# # Her bir merchant_id için aykırı değerleri tespit etmek, baskılamak ve sonuçları görmek
# merchant_ids = model_2023_df['merchant_id'].unique()
# capped_results = []  # Sonuçları saklamak için boş bir liste

# for merchant_id in merchant_ids:
#     merchant_df = model_2023_df[model_2023_df['merchant_id'] == merchant_id].copy()
#     capped_df = detect_and_cap_outliers(merchant_df)
#     capped_results.append(capped_df)

# # Sonuçların birleştirilmesi
# model_2023_df = pd.concat(capped_results).reset_index(drop=True)
# model_2023_df = model_2023_df[['merchant_id', 'month_id' ,'net_payment_count']]

# # İlk birkaç sonucu göster
# model_2023_df.head()

In [43]:
model_2023_df.net_payment_count.mean()

97.36906510034264

In [44]:
# Gözlem sayılarını hesaplama
merchant_observation_counts = model_2023_df['merchant_id'].value_counts()

# 11'den fazla gözlem içeren merchant'ları belirleme
merchants_more_than_11 = merchant_observation_counts[merchant_observation_counts >= 8].index

# 11'den az gözlem içeren merchant'ları belirleme
merchants_less_than_11 = merchant_observation_counts[merchant_observation_counts < 8].index

# Bu merchant'ların gözlemlerini ilgili DataFrame'lere ayırma
model_2023_df_up = model_2023_df[model_2023_df['merchant_id'].isin(merchants_more_than_11)]
model_2023_df_down = model_2023_df[model_2023_df['merchant_id'].isin(merchants_less_than_11)]

# Sonuçların boyutlarını kontrol etme
model_2023_df_up.shape, model_2023_df_down.shape

((1799, 3), (8416, 3))

In [45]:
unique_merchant_ids = churn_df['merchant_id'].unique()

# Her merchant_id için 2023-10, 2023-11, ve 2023-12 tarihleri için id oluştur
submission_entries = []
for merchant_id in unique_merchant_ids:
    for month in ['10', '11', '12']:
        submission_id = f'2023{month}{merchant_id}'
        submission_entries.append([submission_id, merchant_id, None])

# sub_no_model DataFrame'ini oluştur
churn = pd.DataFrame(submission_entries, columns=['id', 'merchant_id', 'net_payment_count'])

churn.head(20)

,id,merchant_id,net_payment_count
0,202310merchant_10026,merchant_10026,None
1,202311merchant_10026,merchant_10026,None
2,202312merchant_10026,merchant_10026,None
3,202310merchant_10116,merchant_10116,None
4,202311merchant_10116,merchant_10116,None
5,202312merchant_10116,merchant_10116,None
6,202310merchant_10160,merchant_10160,None
7,202311merchant_10160,merchant_10160,None
8,202312merchant_10160,merchant_10160,None
9,202310merchant_10189,merchant_10189,None


In [46]:
churn['net_payment_count'] = 0
churn = churn[['id', 'net_payment_count']]
churn.head(3)

,id,net_payment_count
0,202310merchant_10026,0
1,202311merchant_10026,0
2,202312merchant_10026,0


In [47]:
def filler(df):
    # Define the target date
    target_date = pd.to_datetime('2023-09-01')

    # Find merchants that don't have a record on the target date
    merchants_missing_target_date = df[~df['merchant_id'].isin(
        df[df['month_id'] == target_date]['merchant_id']
    )]['merchant_id'].unique()

    # Get the first record of each merchant to preserve the static features
    first_records_per_merchant = df[df['merchant_id'].isin(merchants_missing_target_date)].groupby('merchant_id').first().reset_index()

    # Create missing records for the target date
    missing_records = first_records_per_merchant.copy()
    missing_records['month_id'] = target_date
    missing_records['net_payment_count'] = 0

    # Append the missing records to the original DataFrame
    df = pd.concat([df, missing_records], ignore_index=True)

    # Sort the updated DataFrame
    df.sort_values(by=['merchant_id', 'month_id'], inplace=True)
    
    return df
  
model_2023_df_up = filler(model_2023_df_up)  
model_2023_df_down = filler(model_2023_df_down)  

In [48]:
static_features_df = static_df.drop_duplicates()

In [49]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

model_2023_up_data = TimeSeriesDataFrame.from_data_frame(
    model_2023_df_up,
    id_column="merchant_id",
    timestamp_column="month_id",
    static_features_df=static_features_df
)
model_2023_up_data.head()

net_payment_count
item_id      timestamp                    
merchant_101 2023-02-01                 19
             2023-03-01                 31
             2023-04-01                 26
             2023-05-01                 19
             2023-06-01                 21

In [50]:
model_2023_up_data = model_2023_up_data.convert_frequency(freq="M")

In [51]:
model_2023_up_data['net_payment_count'] = model_2023_up_data['net_payment_count'].fillna(0)

In [52]:
from autogluon.common import space

predictor = TimeSeriesPredictor(
    prediction_length=3,
    target="net_payment_count",
    eval_metric="MAE",
    freq='M',
    quantile_levels=[0.3, 0.31, 0.32, 0.33, 0.34 ,0.35, 0.4, 0.5],
#     known_covariates_names=["month", "year"]
)

predictor.fit(
    model_2023_up_data,
    presets="best_quality",
    time_limit= 3600 * 6,
    refit_full=True
    # excluded_model_types=["AutoARIMA", "DeepAR", "CrostonSBA", "NPTS", "DirectTabular", 
    #                      "TemporalFusionTransformer", "PatchTST"],
    
#     hyperparameters={
#       "SeasonalNaive": {"n_jobs": 6},
#     #   "Naive": {"n_jobs": 6},
#     #   "AutoETS": {"n_jobs": 6},
#       "DynamicOptimizedTheta": {"n_jobs": 6},
#       "RecursiveTabular": {"n_jobs": 6},
#     #   "AutoCES": {"n_jobs": 6},
# #       "ADIDA": {},
# #       "IMAPA": {},
# #       "DLinear": {},
# #       "SimpleFeedForward": {},
# #       "DeepAR": {},
#         # "TemporalFusionTransformer": {"n_jobs": 6},
#         },
#     hyperparameter_tune_kwargs={
# #     "num_trials": 5,
# #     "scheduler": "local",
# #     "searcher": "random",
#     "n_jobs": -1
# },
)

Beginning AutoGluon training... Time limit = 21600s
AutoGluon will save models to 'AutogluonModels\ag-20240225_084235'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
GPU Count:          0
Memory Avail:       3.84 GB / 15.42 GB (24.9%)
Disk Space Avail:   31.36 GB / 476.34 GB (6.6%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'freq': 'M',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 3,
 'quantile_levels': [0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.4, 0.5],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': True,
 'target': 'net_payment_count',
 'time_limit': 21600,
 'verbosity': 2}

Provided train_data has 1828 rows, 214 time series. Median time series length is 9 (min=8, max=9). 
Time series

In [53]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-40.307,37.069,1.338,12
1,TemporalFusionTransformer,-56.102,0.391,183.048,10
2,DynamicOptimizedTheta,-57.617,29.845,0.024,5
3,SeasonalNaive,-71.833,6.618,0.020,1
4,NPTS,-71.833,0.333,0.023,3
5,AutoETS,-71.833,0.734,0.022,4
6,AutoARIMA,-90.976,34.183,0.016,6
7,DeepAR,-102.242,0.638,82.346,9
8,RecursiveTabular,-137.454,0.215,3.070,7
9,PatchTST,-161.358,0.106,91.409,11


In [54]:
predictions_up = predictor.predict(model_2023_up_data, model='WeightedEnsemble')

In [55]:
results = predictions_up.copy().reset_index()
results['id'] = results['timestamp'].dt.strftime('%Y%m') + results['item_id']

# Select the 'id' and 'mean' columns and rename 'mean' to 'net_payment_count'
model_2023_up_sub = results[['id', '0.4']].rename(columns={'0.4': 'net_payment_count'})

In [56]:
model_2023_up_sub.to_csv('model_2023_up_sub.csv', index=False)

In [57]:
churn.to_csv('churn_2023.csv', index=False)

In [58]:
static_features_df = static_df.drop_duplicates()

In [59]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

model_2023_down_data = TimeSeriesDataFrame.from_data_frame(
    model_2023_df_down,
    id_column="merchant_id",
    timestamp_column="month_id",
    static_features_df=static_features_df
)
model_2023_down_data.head()

net_payment_count
item_id        timestamp                    
merchant_10003 2023-07-01                  3
               2023-08-01                  3
               2023-09-01                  0
merchant_10006 2023-01-01                  3
               2023-04-01                  3

In [60]:
model_2023_down_data = model_2023_down_data.convert_frequency(freq="M")
model_2023_down_data['net_payment_count'] = model_2023_down_data['net_payment_count'].fillna(0)

In [61]:
from autogluon.common import space

predictor = TimeSeriesPredictor(
    prediction_length=3,
    target="net_payment_count",
    eval_metric="MAE",
    freq='M',
    quantile_levels=[0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7],
#     known_covariates_names=["month", "year"]
)

predictor.fit(
    model_2023_down_data,
    presets="best_quality",
    time_limit= 3600 * 6,
    excluded_model_types=["AutoARIMA", "DeepAR", "CrostonSBA", "NPTS", "DirectTabular", 
                         "TemporalFusionTransformer", "PatchTST"],
    refit_full=True,
    
#     hyperparameters={
#       "SeasonalNaive": {"n_jobs": 6},
#       "Naive": {"n_jobs": 6},
#       "AutoETS": {"n_jobs": 6},
#       "DynamicOptimizedTheta": {"n_jobs": 6},
#       "RecursiveTabular": {"n_jobs": 6},
#       "AutoCES": {"n_jobs": 6},
# #       "ADIDA": {},
# #       "IMAPA": {},
# #       "DLinear": {},
# #       "SimpleFeedForward": {},
# #       "DeepAR": {},
#         },
#     hyperparameter_tune_kwargs={
# #     "num_trials": 5,
# #     "scheduler": "local",
# #     "searcher": "random",
#     "n_jobs": -1
# },
)

Beginning AutoGluon training... Time limit = 21600s
AutoGluon will save models to 'AutogluonModels\ag-20240225_085048'
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.10
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          12
GPU Count:          0
Memory Avail:       3.16 GB / 15.42 GB (20.5%)
Disk Space Avail:   31.34 GB / 476.34 GB (6.6%)
Setting presets to: best_quality

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MAE,
 'excluded_model_types': ['AutoARIMA',
                          'DeepAR',
                          'CrostonSBA',
                          'NPTS',
                          'DirectTabular',
                          'TemporalFusionTransformer',
                          'PatchTST'],
 'freq': 'M',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 3,
 'quantile_levels': [0.2, 0.25, 0.3

In [62]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-3.937,4.494,0.507,5
1,SeasonalNaive,-3.937,4.494,0.015,1
2,AutoETS,-3.937,20.938,0.016,2
3,DynamicOptimizedTheta,-4.397,26.898,0.013,3
4,RecursiveTabular,-6.266,0.875,3.264,4
5,WeightedEnsemble_FULL,NaN,NaN,0.507,10
6,SeasonalNaive_FULL,NaN,NaN,0.000,6
7,RecursiveTabular_FULL,NaN,NaN,3.249,9
8,DynamicOptimizedTheta_FULL,NaN,NaN,0.000,8
9,AutoETS_FULL,NaN,NaN,0.000,7


In [63]:
predictions = predictor.predict(model_2023_down_data, model='WeightedEnsemble')

In [64]:
predictions.head(5)

mean    0.2   0.25    0.3   0.35    0.4   0.45   0.5   0.6   0.7
item_id        timestamp                                                                   
merchant_10003 2023-10-31 0.000 -1.785 -1.431 -1.112 -0.817 -0.537 -0.267 0.000 0.537 1.112
               2023-11-30 0.000 -2.525 -2.023 -1.573 -1.156 -0.760 -0.377 0.000 0.760 1.573
               2023-12-31 0.000 -3.092 -2.478 -1.927 -1.416 -0.931 -0.462 0.000 0.931 1.927
merchant_10006 2023-10-31 3.000  1.215  1.569  1.888  2.183  2.463  2.733 3.000 3.537 4.112
               2023-11-30 3.000  0.475  0.977  1.427  1.844  2.240  2.623 3.000 3.760 4.573

In [65]:
results = predictions.copy().reset_index()
results['id'] = results['timestamp'].dt.strftime('%Y%m') + results['item_id']

# Select the 'id' and 'mean' columns and rename 'mean' to 'net_payment_count'
model_2023_down_sub = results[['id', '0.5']].rename(columns={'0.5': 'net_payment_count'})

In [66]:
model_2023_down_sub = model_2023_down_sub.fillna(2)

In [67]:
model_2023_down_sub.to_csv('model_2023_down_sub.csv', index=False)

In [68]:
model_2023_down_sub.isnull().sum(), model_2023_up_sub.isnull().sum()

(id                   0
 net_payment_count    0
 dtype: int64,
 id                   0
 net_payment_count    0
 dtype: int64)